In [15]:
import numpy as np
import pandas as pd
import re
import time
import spacy as sp
import string
import os
import en_core_web_sm
import pandas as pd
import gensim
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
import urllib.request
nlp = sp.load('en_core_web_sm')

In [16]:
def prepossessing(comments):
    processed_comments = []
    totalSentences = 0
    totalTokens = 0
    totalWords = 0
    numComments = len(comments)
    
    for comment in comments:
        # Process the comment using spaCy NLP model
        doc = nlp(comment)
        
        # Count the number of sentences
        totalSentences += len(list(doc.sents))
        
        # Count the number of tokens
        totalTokens += len(doc)
        
        # Count the number of words without considering punctuation
        tokens = comment.split()
        tokens = [token for token in tokens if token not in string.punctuation]
        totalWords += len(tokens)   
        
        processed_comments.append(doc)
        
    avgSentences = totalSentences / len(comments)
    avgTokens = totalTokens / len(comments)
    avgWords = totalWords / len(comments)
    
    print("Average number of sentences per comment:", avgSentences)
    print("Average number of tokens per comment:", avgTokens)
    print("Average number of words per comment (without considering punctuation):", avgWords)
    return avgSentences,avgTokens,avgWords

In [17]:
def train_word2vec(i,words):
    startTime = time.time()
    model = Word2Vec(words, vector_size=i,window = 5, min_count = 1,workers=4)
    endTime = time.time()
    print('Time used for training',endTime-startTime,'Seconds')
    print('Computer Parameters: ')
    print(f"- CPU: {os.cpu_count()} cores") 
    print("Vocabulary size:", len(model.wv.index_to_key))
    if(i == 1):
        model.save('model/word2vec1.model')
    if(i==10):
        model.save('model/word2vec10.model')
    if(i==100):
        model.save('model/word2vec100.model')

In [18]:
def evaluate_word2vec(i,word):
    if(i == 1):
        model = Word2Vec.load('model/word2vec1.model')
    if(i == 10):
        model = Word2Vec.load('model/word2vec10.model')
    if(i == 100):
        model = Word2Vec.load('model/word2vec100.model') 
    vector = model.wv[word]
    similar_words = model.wv.most_similar(word,topn = 10)
    print(similar_words)

In [19]:
def train_GloVe():
    #pretrained GloVe model
    glove_file = 'glove/vectors.txt'
    tmp_file = "glove_To_word2vec.txt"
    #Convert word2vec model to GloVe model
    f = glove2word2vec(glove_file, tmp_file)
    model = KeyedVectors.load_word2vec_format(tmp_file)
    model.save('model/glove.model')

In [20]:
def evaluate_glove(word):
	model = KeyedVectors.load('model/glove.model')
	vector = model[word]
	similar_words = model.most_similar(word,topn = 10)
	print(similar_words)

In [21]:
directory = "comments1k"
comments = []
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open(os.path.join(directory, filename)) as file:
            comments.append(file.read().strip())
df1 = pd.DataFrame(comments)

df1[0] = df1[0].str.replace('&\w+;','')
df1[0] = df1[0].apply(lambda x: re.sub('<.*?>', '', x))

df1[0] = df1[0].str.lower()

df1[0] = df1[0].str.replace('[^\w\s]','')

df = [row.split(' ') for row in df1[0]]


C:\Users\saima\AppData\Local\Temp\ipykernel_13404\905427216.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df1[0] = df1[0].str.replace('&\w+;','')
C:\Users\saima\AppData\Local\Temp\ipykernel_13404\905427216.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df1[0] = df1[0].str.replace('[^\w\s]','')


In [22]:
df1

,0
0,bromwell high is a cartoon comedy it ran at th...
1,scott bartletts offon is nine minutes of pure ...
2,imdb lists this as 1972 for some reason but th...
3,i first heard about this film about 20 years a...
4,when i read other commenti decided to watch th...
...,...
991,i have to agree with most of the other posts w...
992,this is a really interesting movie it is an ac...
993,i am amazed at how this movieand most others h...
994,a christmas together actually came before my t...


In [23]:
if __name__ == "__main__":
    prepossessing(comments)
    print('word2vec method with CBOW model and vector size as 100')
    train_word2vec(100,df)
    print('model  trained  in word2vec of 100 vector size by  analyzing  the  10  most  similar  words of movie :')
    evaluate_word2vec(100,'movie')
    print('model  trained  in word2vec of 100 vector size by  analyzing  the  10  most  similar  words of music :')
    evaluate_word2vec(100,'music')
    print('model  trained  in word2vec of 100 vector size by  analyzing  the  10  most  similar  words of woman :')
    evaluate_word2vec(100,'woman')
    print('model  trained  in word2vec of 100 vector size by  analyzing  the  10  most  similar  words of Christmas :')
    evaluate_word2vec(100,'christmas')
    print('\n')
    print('Glove model and vector size as 100')
    train_GloVe()
    print('model  trained  in glove by  analyzing  the  10  most  similar  words of movie :')
    evaluate_glove('movie')
    print('model  trained  in glove by  analyzing  the  10  most  similar  words of music :')
    evaluate_glove('music')
    print('model  trained  in glove by  analyzing  the  10  most  similar  words of woman :')
    evaluate_glove('woman')
    print('model  trained  in glove by  analyzing  the  10  most  similar  words of Christmas :')
    evaluate_glove('christmas')

Average number of sentences per comment: 13.301204819277109
Average number of tokens per comment: 272.5301204819277
Average number of words per comment (without considering punctuation): 234.26706827309238
word2vec method with CBOW model and vector size as 100
Time used for training 0.9705944061279297 Seconds
Computer Parameters: 
- CPU: 8 cores
Vocabulary size: 19272
model  trained  in word2vec of 100 vector size by  analyzing  the  10  most  similar  words of movie :
[('film', 0.9733423590660095), ('doubt', 0.9663307666778564), ('thing', 0.9636635184288025), ('saw', 0.9636475443840027), ('liked', 0.962794303894043), ('recommend', 0.9621709585189819), ('thought', 0.9605566263198853), ('comment', 0.9585217833518982), ('felt', 0.9579874277114868), ('bought', 0.9572857618331909)]
model  trained  in word2vec of 100 vector size by  analyzing  the  10  most  similar  words of music :
[('war', 0.9993916749954224), ('character', 0.999042272567749), ('novel', 0.9990364909172058), ('supporting'

C:\Users\saima\AppData\Local\Temp\ipykernel_13404\1833267865.py:6: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  f = glove2word2vec(glove_file, tmp_file)


model  trained  in glove by  analyzing  the  10  most  similar  words of movie :
[('this', 0.8925120830535889), ('film', 0.8462427854537964), ('it', 0.7992120385169983), ('made', 0.7556362152099609), ('actually', 0.7472920417785645), ('think', 0.7353578209877014), ('just', 0.7298932075500488), ('really', 0.7049843668937683), ('one', 0.7032972574234009), ('but', 0.6959174871444702)]
model  trained  in glove by  analyzing  the  10  most  similar  words of music :
[('dialogue', 0.772396981716156), ('great', 0.7340028882026672), ('effects', 0.722646176815033), ('video', 0.7005497813224792), ('cinematography', 0.6866236329078674), ('special', 0.6834750771522522), ('scenery', 0.6818291544914246), ('casting', 0.6797624826431274), ('touching', 0.677040696144104), ('dramatic', 0.6764266490936279)]
model  trained  in glove by  analyzing  the  10  most  similar  words of woman :
[('girl', 0.831171989440918), ('named', 0.8100281953811646), ('young', 0.8018996119499207), ('man', 0.7678036093711853)

In [24]:
print('word2vec method with CBOW model and vector size as 1')
train_word2vec(1,df)

word2vec method with CBOW model and vector size as 1
Time used for training 1.5334162712097168 Seconds
Computer Parameters: 
- CPU: 8 cores
Vocabulary size: 19272


In [25]:
print('model  trained  in word2vec of 1 vector size by  analyzing  the  10  most  similar  words of movie :')
evaluate_word2vec(1,'movie')
print('model  trained  in word2vec of 1 vector size by  analyzing  the  10  most  similar  words of music :')
evaluate_word2vec(1,'music')
print('model  trained  in word2vec of 1 vector size by  analyzing  the  10  most  similar  words of woman :')
evaluate_word2vec(1,'woman')
print('model  trained  in word2vec of 1 vector size by  analyzing  the  10  most  similar  words of Christmas :')
evaluate_word2vec(1,'christmas')

model  trained  in word2vec of 1 vector size by  analyzing  the  10  most  similar  words of movie :
[('missions', 1.0), ('wholeheartedlywell', 1.0), ('jugular', 1.0), ('sideline', 1.0), ('stara', 1.0), ('ingãnue', 1.0), ('garbo', 1.0), ('onereel', 1.0), ('manshow', 1.0), ('collaborate', 1.0)]
model  trained  in word2vec of 1 vector size by  analyzing  the  10  most  similar  words of music :
[('missions', 1.0), ('wholeheartedlywell', 1.0), ('jugular', 1.0), ('sideline', 1.0), ('stara', 1.0), ('ingãnue', 1.0), ('garbo', 1.0), ('onereel', 1.0), ('manshow', 1.0), ('collaborate', 1.0)]
model  trained  in word2vec of 1 vector size by  analyzing  the  10  most  similar  words of woman :
[('missions', 1.0), ('wholeheartedlywell', 1.0), ('jugular', 1.0), ('sideline', 1.0), ('stara', 1.0), ('ingãnue', 1.0), ('garbo', 1.0), ('onereel', 1.0), ('manshow', 1.0), ('collaborate', 1.0)]
model  trained  in word2vec of 1 vector size by  analyzing  the  10  most  similar  words of Christmas :
[('mission

In [26]:
print('word2vec method with CBOW model and vector size as 10')
train_word2vec(10,df)

word2vec method with CBOW model and vector size as 10
Time used for training 1.5839741230010986 Seconds
Computer Parameters: 
- CPU: 8 cores
Vocabulary size: 19272


In [27]:
print('model  trained  in word2vec of 10 vector size by  analyzing  the  10  most  similar  words of movie :')
evaluate_word2vec(10,'movie')
print('model  trained  in word2vec of 10 vector size by  analyzing  the  10  most  similar  words of music :')
evaluate_word2vec(10,'music')
print('model  trained  in word2vec of 10 vector size by  analyzing  the  10  most  similar  words of woman :')
evaluate_word2vec(10,'woman')
print('model  trained  in word2vec of 10 vector size by  analyzing  the  10  most  similar  words of Christmas :')
evaluate_word2vec(10,'christmas')

model  trained  in word2vec of 10 vector size by  analyzing  the  10  most  similar  words of movie :
[('film', 0.9873573780059814), ('saw', 0.9842372536659241), ('taste', 0.9759257435798645), ('tape', 0.9733883738517761), ('thing', 0.9716353416442871), ('sit', 0.9695032238960266), ('loved', 0.9674909710884094), ('recommend', 0.965630829334259), ('overthetop', 0.9645233750343323), ('crashing', 0.9614243507385254)]
model  trained  in word2vec of 10 vector size by  analyzing  the  10  most  similar  words of music :
[('direction', 0.9991080164909363), ('killer', 0.9979554414749146), ('themes', 0.997456431388855), ('magic', 0.9974030256271362), ('clever', 0.9973888993263245), ('action', 0.9973253011703491), ('excellent', 0.9972029328346252), ('especially', 0.996849536895752), ('place', 0.9968379735946655), ('late', 0.9966833591461182)]
model  trained  in word2vec of 10 vector size by  analyzing  the  10  most  similar  words of woman :
[('young', 0.9986621141433716), ('boy', 0.99732911586

In [35]:
print('model  trained  in word2vec of 100 vector size by  analyzing  the  10  most  similar  words of movie :')
evaluate_word2vec(100,'movie')
print('model  trained  in word2vec of 100 vector size by  analyzing  the  10  most  similar  words of music :')
evaluate_word2vec(100,'music')
print('model  trained  in word2vec of 100 vector size by  analyzing  the  10  most  similar  words of woman :')
evaluate_word2vec(100,'woman')
print('model  trained  in word2vec of 100 vector size by  analyzing  the  10  most  similar  words of Christmas :')
evaluate_word2vec(100,'christmas')

model  trained  in word2vec of 100 vector size by  analyzing  the  10  most  similar  words of movie :
[('film', 0.9733423590660095), ('doubt', 0.9663307666778564), ('thing', 0.9636635184288025), ('saw', 0.9636475443840027), ('liked', 0.962794303894043), ('recommend', 0.9621709585189819), ('thought', 0.9605566263198853), ('comment', 0.9585217833518982), ('felt', 0.9579874277114868), ('bought', 0.9572857618331909)]
model  trained  in word2vec of 100 vector size by  analyzing  the  10  most  similar  words of music :
[('war', 0.9993916749954224), ('character', 0.999042272567749), ('novel', 0.9990364909172058), ('supporting', 0.9990068078041077), ('picture', 0.9989712834358215), ('jedi', 0.9988833069801331), ('beautiful', 0.998854398727417), ('beauty', 0.9988070130348206), ('wonderful', 0.9987823367118835), ('early', 0.9987727403640747)]
model  trained  in word2vec of 100 vector size by  analyzing  the  10  most  similar  words of woman :
[('man', 0.9994759559631348), ('young', 0.99911856